* 덧셈 데이터 구성
* seq2seq Encoder / Decoder 정의
* 학습
* 평가 (예측)

## 덧셈 데이터 구성

In [ ]:
# 최대 세자리수끼리 합을 구하는 데이터

In [ ]:
import numpy as np

In [ ]:
# 99 + 1 => '9','9','+','1'

In [ ]:
def get_sum_dict():
  items = list(str(x) for x in range(10))
  items.append('_')
  items.append('+')

  id_to_item = {}
  for idx, item in enumerate(items):
    id_to_item[idx]=item
  
  item_to_id = dict([(value, key) for key, value in id_to_item.items()])

  return id_to_item, item_to_id

In [ ]:
def convert_item_to_id(items, item_to_id):
  ids = list([item_to_id[item] for item in items])
  return ids

def convert_id_to_item(ids, id_to_item):
  items = list([id_to_item[id] for id in ids])
  return items

In [ ]:
def sum_data_gen():
  num1 = np.random.randint(0,1000)
  num2 = np.random.randint(0,1000)
  ans = num1 + num2
  q_str = str(num1) + '+' + str(num2)
  a_str = '_' + str(ans).zfill(4)
  return list(q_str), list(a_str)

In [ ]:
def get_dataset(data_num=100):
  id_to_item, item_to_id = get_sum_dict()
  xs = []
  ys = []
  for i in range(data_num):
    x, y = sum_data_gen()
    xs.append(convert_item_to_id(x, item_to_id))
    ys.append(convert_item_to_id(y, item_to_id))
  return xs, ys   

In [ ]:
train_x, train_y = get_dataset(data_num=20000)
test_x, test_y = get_dataset(data_num=10000)
id_to_item, item_to_id = get_sum_dict()

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
src_vocab_size = len(id_to_item) 
tar_vocab_size = len(id_to_item)

In [ ]:
train_x = pad_sequences(train_x, maxlen=7, padding='pre')
test_x = pad_sequences(test_x, maxlen=7, padding='pre')
train_y = pad_sequences(train_y, maxlen=5, padding='pre')
test_y = pad_sequences(test_y, maxlen=5, padding='pre')

In [ ]:
train_y_t = train_y[:,1:]
test_y_t = test_y[:,1:]

train_y_t = pad_sequences(train_y_t, maxlen=5, padding='pre')
test_y_t = pad_sequences(test_y_t, maxlen=5, padding='pre')

In [ ]:
train_x = to_categorical(train_x, num_classes=vocab_size)
test_x = to_categorical(test_x, num_classes=vocab_size)
train_y = to_categorical(train_y, num_classes=vocab_size)
test_y = to_categorical(test_y, num_classes=vocab_size)
train_y_t = to_categorical(train_y_t, num_classes=vocab_size)
test_y_t = to_categorical(test_y_t, num_classes=vocab_size)

In [ ]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape, train_y_t.shape, test_y_t.shape

((20000, 7, 12),
 (10000, 7, 12),
 (20000, 5, 12),
 (10000, 5, 12),
 (20000, 5, 12),
 (10000, 5, 12))

In [ ]:
train_y_t[:2]

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]], dtype=float32)

In [ ]:
# Encoder 
# encoder_input = Embedding(vocab_size, 5,  input_length=7)
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(8, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
# Decoder
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(8, return_state=True, return_sequences=True)
# output, h, c 자리인데, decoder의 h,c는 사용되지 않기 때문에 _ 로..
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

# Model(input, output)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, 8), (None, 8 672         input_8[0][0]                    
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, None, 8), (N 672         input_9[0][0]                    
                                                                 lstm_10[0][1]         

In [ ]:
# train_x.shape, test_x.shape, 
# train_y.shape, test_y.shape, 
# train_y_t.shape, test_y_t.shape

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy') 

In [ ]:
model.fit(x=[train_x, train_y], 
              y=train_y_t, 
              batch_size=512, 
              epochs=50, 
              validation_split=0.2)             

Epoch 1/50
32/32 [==============================] - 1s 36ms/step - loss: 2.4288 - val_loss: 2.3677
Epoch 2/50
32/32 [==============================] - 0s 13ms/step - loss: 2.2886 - val_loss: 2.1835
Epoch 3/50
32/32 [==============================] - 0s 12ms/step - loss: 2.0813 - val_loss: 1.9781
Epoch 4/50
32/32 [==============================] - 0s 12ms/step - loss: 1.8984 - val_loss: 1.8148
Epoch 5/50
32/32 [==============================] - 0s 12ms/step - loss: 1.7484 - val_loss: 1.6806
Epoch 6/50
32/32 [==============================] - 0s 12ms/step - loss: 1.6261 - val_loss: 1.5695
Epoch 7/50
32/32 [==============================] - 0s 13ms/step - loss: 1.5203 - val_loss: 1.4677
Epoch 8/50
32/32 [==============================] - 0s 12ms/step - loss: 1.4192 - val_loss: 1.3667
Epoch 9/50
32/32 [==============================] - 0s 12ms/step - loss: 1.3172 - val_loss: 1.2641
Epoch 10/50
32/32 [==============================] - 0s 12ms/step - loss: 1.2139 - val_loss: 1.1605
Epoch 11/